# tomoDRGN interactive visualization and filtering

This jupyter notebook provides multiple methods for filtering particles based on the latent space, including:
* clustering of the latent space (k-means or Gaussian mixture model)
* outlier detection (Z-score)
* interactive selection with a lasso tool

For each method, the selected particles are tracked in the variable, `ind_selected`.

Selected particles can be analyzed for correlations with other outputs from tomoDRGN training/analysis, and/or spatial context from the final M starfile

Once the selection has been finalized, the selected particles are saved as a `index.pkl` file at the end of this notebook. The selected indices can be provided to tomoDRGN with the `--ind` argument to train a new model on a subset of the images or converted to `.star` file format.

For full functionality, the following files are required:
* tomoDRGN train_vae_ts training run outputs (.pkl)
* tomoDRGN analyze outputs (.pkl)
* M starfile from the final round of traditional STA alignment (.star)
* reconstructed tomograms (.mrc)

In [ ]:
import pandas as pd
import numpy as np
import pickle
import subprocess
import os, sys

from cryodrgn import analysis
from cryodrgn import utils
from cryodrgn import dataset
from cryodrgn import ctf
from cryodrgn import starfile
                
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
from ipywidgets import interact, interactive, HBox, VBox
from scipy.spatial.transform import Rotation as RR
py.init_notebook_mode()
from IPython.display import FileLink, FileLinks

In [ ]:
# Enable interactive widgets
!jupyter nbextension enable --py widgetsnbextension

### Load results

In [ ]:
# Specify the workdir and the epoch number (0-based index) to analyze
WORKDIR = '..' 
EPOCH = 49 # CHANGE ME

In [ ]:
print(os.path.abspath(WORKDIR))

In [ ]:
# Load z
with open(f'{WORKDIR}/z.{EPOCH}.pkl','rb') as f:
    z = pickle.load(f)
    z_logvar = pickle.load(f)

In [ ]:
# Load UMAP
umap = utils.load_pkl(f'{WORKDIR}/analyze.{EPOCH}/umap.pkl')
# or run UMAP
# umap = analysis.run_umap(z)

In [ ]:
# Load kmeans
K = 20
kmeans_labels = utils.load_pkl(f'{WORKDIR}/analyze.{EPOCH}/kmeans{K}/labels.pkl')
kmeans_centers = np.loadtxt(f'{WORKDIR}/analyze.{EPOCH}/kmeans{K}/centers.txt')
# Or re-run kmeans with the desired number of classes
#kmeans_labels, kmeans_centers = analysis.cluster_kmeans(z, 20)

# Get index for on-data cluster center
kmeans_centers, centers_ind = analysis.get_nearest_point(z, kmeans_centers)

### Load dataset

In [ ]:
# Load configuration file
config = utils.load_pkl(f'{WORKDIR}/config.pkl')
print(config)

In [ ]:
# Load poses
pose_pkl = config['dataset_args']['poses']
rot, trans = utils.load_pkl(pose_pkl)

In [ ]:
# Convert rotation matrices to euler angles
euler = RR.from_matrix(rot).as_euler('zyz', degrees=True)

In [ ]:
# Load ptcl index filter
ind_ptcl_filt = config['dataset_args']['ind']
if ind_ptcl_filt is not None:
    ind_ptcl_filt = utils.load_pkl(ind_ptcl_filt)

# Load image index filter
ind_img_filt = config['dataset_args']['expanded_ind']

# apply index filter to poses
if len(rot) > len(ind_img_filt):
    print(f'Filtering poses from {len(rot)} to {len(ind_img_filt)}')
    rot = rot[ind_img_filt]
    trans = trans[ind_img_filt]
    euler = euler[ind_img_filt]

In [ ]:
# Load input particles
particles = dataset.load_particles(config['dataset_args']['particles'],
                            lazy=True,
                            datadir=config['dataset_args']['datadir'])
n_ptcl_orig = len(particles)
if len(particles) > len(ind_img_filt):
    print(f'Filtering particles from {len(particles)} to {len(ind_img_filt)}')
    particles = [particles[i] for i in ind_img_filt]

In [ ]:
# Load CTF
ctf_params = utils.load_pkl(config['dataset_args']['ctf'])
if len(ctf_params) > len(ind_img_filt):
    print(f'Filtering ctf parameters from {len(ctf_params)} to {len(ind_img_filt)}')
    ctf_params = ctf_params[ind_img_filt]
ctf.print_ctf_params(ctf_params[0])

In [ ]:
# Load additional metadata from M outfile (tomogram positions, tomogram IDs)
path_to_M_outfile = '' # USER INPUT: INSERT ABSOLUTE PATH TO M STARFILE
mstar = starfile.GenericStarfile(path_to_M_outfile)
df_mstar = mstar.blocks['data_']
if ind_ptcl_filt is not None:
    df_mstar = df_mstar[df_mstar.index.isin(ind_ptcl_filt)]

# Assign unique, sequential ID numbers to each tomogram for later visualization
ind_tomo = {tomo_name : tomo_index for tomo_index, tomo_name in enumerate(df_mstar['_rlnMicrographName'].unique())}
df_mstar.loc[:,'ind_tomo'] = [ind_tomo[row] for row in df_mstar['_rlnMicrographName']]

# rescale df xyz coordinates to match reconstructed tomograms pixel size
df_mstar = analysis.rescale_df_coordinates(df_mstar, tomo_max_xyz_nm=(680, 680, 510), tomo_pixelsize=10, starfile_pixelsize=1)
df_mstar

### Define helper functions

In [ ]:
def list_paths_by_extension(basedir, extension):
    file_list = [os.path.join(basedir, file) for file in os.listdir(basedir) if file.endswith(extension)]
    return sorted(file_list)

In [ ]:
def invert_selection(all_labels, ind_selected):
    return np.array(sorted(set(np.arange(len(all_labels))) - set(ind_selected)))

In [ ]:
def combine_selection(ind_sel1, ind_sel2, kind='union'):
    assert kind in ('union','intersection')
    return analysis.combine_ind(len(z), ind_sel1, ind_sel2, kind)

In [ ]:
def select_clusters(labels, cluster_ids):
    return analysis.get_ind_for_cluster(labels, cluster_ids)

In [ ]:
def save_selection(path, index):
    if ind_ptcl_filt is not None:
        print('Converting to original .mrcs indices')
        index = analysis.convert_original_indices(index, n_ptcl_orig, ind_ptcl_filt)
    utils.save_pkl(index, path)
    print(f'Wrote {os.path.abspath(path)}')

# Plot dataset / model properties

### View pose distribution

In [ ]:
# rotations
analysis.plot_euler(euler[:,0],euler[:,1], euler[:,2])

In [ ]:
# translations
# set near-zero translations to 0.0 to allow sns.jointplot to work
trans[np.isclose(trans, 0.0, atol=1e-4)] = 0.0
sns.jointplot(x=trans[:,0],
              y=trans[:,1],
              kind='hex').set_axis_labels('tx (fraction)','ty (fraction)')

### Learning curve

In [ ]:
loss = analysis.parse_loss(f'{WORKDIR}/run.log')
plt.plot(loss)
plt.xlabel('epoch')
plt.ylabel('loss')

### PCA

In [ ]:
pc, pca = analysis.run_pca(z)

In [ ]:
g = sns.jointplot(x=pc[:,0], y=pc[:,1], alpha=.1, s=1)
g.set_axis_labels('PC1', 'PC2')

In [ ]:
g = sns.jointplot(x=pc[:,0], y=pc[:,1], kind='hex')
g.set_axis_labels('PC1', 'PC2')

In [ ]:
plt.bar(np.arange(z.shape[1])+1,pca.explained_variance_ratio_)
plt.xticks(np.arange(z.shape[1])+1)
plt.xlabel('PC')
plt.ylabel('explained variance')

### UMAP

In [ ]:
g = sns.jointplot(x=umap[:,0], y=umap[:,1], alpha=.1, s=1)
g.set_axis_labels('UMAP1', 'UMAP2')

In [ ]:
g = sns.jointplot(x=umap[:,0], y=umap[:,1], kind='hex')
g.set_axis_labels('UMAP1', 'UMAP2')

# Filter by clustering

Select particles based on k-means cluster labels or GMM cluster labels

### View K-means clusters

In [ ]:
# Optionally, re-run kmeans with the desired number of classes
#K = 20
#kmeans_labels, kmeans_centers = analysis.cluster_kmeans(z, K)

In [ ]:
K = len(set(kmeans_labels))
c = pca.transform(kmeans_centers) # transform to view with PCs
analysis.plot_by_cluster(pc[:,0], pc[:,1], K, 
                         kmeans_labels, 
                         centers=c,
                         annotate=True)
plt.xlabel('PC1')
plt.ylabel('PC2')

In [ ]:
fig, ax = analysis.plot_by_cluster_subplot(pc[:,0], pc[:,1], K, 
                            kmeans_labels)

In [ ]:
analysis.plot_by_cluster(umap[:,0], umap[:,1], K, 
                         kmeans_labels, 
                         centers_ind=centers_ind,
                         annotate=True)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

In [ ]:
fig, ax = analysis.plot_by_cluster_subplot(umap[:,0], umap[:,1], K, 
                            kmeans_labels)

**Select particles based on k-means clustering**

In [ ]:
cluster_ids = [19] # set clusters to select, 0 and 2 in this example
ind_selected = select_clusters(kmeans_labels, cluster_ids)
ind_selected_not = invert_selection(kmeans_labels, ind_selected)
print('Selected indices:')
print(ind_selected)
print('Number of selected points:')
print(len(ind_selected))
print('Number of unselected points:')
print(len(ind_selected_not))

In [ ]:
# View PCA
plt.scatter(pc[:,0], pc[:,1], alpha=.1, s=1)
plt.scatter(pc[ind_selected,0], pc[ind_selected,1], alpha=.1, s=1)
plt.xlabel('PC1 ({:.2f})'.format(pca.explained_variance_ratio_[0]))
plt.ylabel('PC2 ({:.2f})'.format(pca.explained_variance_ratio_[1]))

In [ ]:
# View umap
plt.scatter(umap[:,0], umap[:,1], alpha=.1, s=1)
plt.scatter(umap[ind_selected,0], umap[ind_selected,1], alpha=.1, s=1)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

### GMM-clustering

In [ ]:
G = 3 # or change to desired cluster number
random_state = np.random.randint(100000) # sample random integer
print(f'Random state: {random_state}')
gmm_labels, gmm_centers = analysis.cluster_gmm(z, G, random_state=random_state)
gmm_centers, gmm_centers_ind = analysis.get_nearest_point(z, gmm_centers)

In [ ]:
analysis.plot_by_cluster(pc[:,0], pc[:,1], G, 
                         gmm_labels, 
                         centers_ind=gmm_centers_ind,
                         annotate=True)
plt.xlabel('PC1')
plt.ylabel('PC2')

In [ ]:
fig, ax = analysis.plot_by_cluster_subplot(pc[:,0], pc[:,1], G, gmm_labels)

In [ ]:
analysis.plot_by_cluster(umap[:,0], umap[:,1], G, 
                         gmm_labels, 
                         centers_ind=gmm_centers_ind,
                         annotate=True)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

In [ ]:
fig, ax = analysis.plot_by_cluster_subplot(umap[:,0], umap[:,1], G, gmm_labels)

**Select particles based on GMM-means clustering**

In [ ]:
cluster_ids = [1] # set clusters to select, 0 and 2 in this example
ind_selected = select_clusters(gmm_labels, cluster_ids)
ind_selected_not = invert_selection(gmm_labels, ind_selected)
print('Selected indices:')
print(ind_selected)
print('Number of selected points:')
print(len(ind_selected))
print('Number of unselected points:')
print(len(ind_selected_not))

In [ ]:
# View PCA
plt.scatter(pc[:,0], pc[:,1], alpha=.1, s=1)
plt.scatter(pc[ind_selected,0], pc[ind_selected,1], alpha=.1, s=1)
plt.xlabel('PC1 ({:.2f})'.format(pca.explained_variance_ratio_[0]))
plt.ylabel('PC2 ({:.2f})'.format(pca.explained_variance_ratio_[1]))

In [ ]:
# View umap
plt.scatter(umap[:,0], umap[:,1], alpha=.1, s=1)
plt.scatter(umap[ind_selected,0], umap[ind_selected,1], alpha=.1, s=1)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

# Filter by latent outliers

In [ ]:
# Compute magnitude of latent encodings
znorm = np.sum(z**2,axis=1)**.5

In [ ]:
# Distribution of ||z||
sns.histplot(znorm, kde=False)
plt.xlabel('||z||')

In [ ]:
# By default, identify particles with ||z|| 2 std deviations above mean
zscore = 2
thresh = znorm.mean()+zscore*znorm.std()
print(f'Mean: {znorm.mean()}, Std: {znorm.std()}, Selected threshold: {thresh}')

In [ ]:
ind_outliers = np.where(znorm >= thresh)[0]
ind_outliers_not = invert_selection(znorm, ind_outliers)

print('Selected indices:')
print(ind_outliers)
print('Number of selected points:')
print(len(ind_outliers))
print('Number of unselected points:')
print(len(ind_outliers_not))

In [ ]:
g = sns.histplot(znorm, kde=False)
plt.axvline(x=thresh)
plt.xlabel('||z||')
plt.title('Magnitude of particle latent encodings')

In [ ]:
# View PCA
plt.scatter(pc[:,0], pc[:,1], alpha=.1, s=1)
plt.scatter(pc[ind_outliers,0], pc[ind_outliers,1], alpha=.1, s=1)
plt.xlabel('PC1 ({:.2f})'.format(pca.explained_variance_ratio_[0]))
plt.ylabel('PC2 ({:.2f})'.format(pca.explained_variance_ratio_[1]))
plt.title(f'Particles with ||z|| > {thresh}')

In [ ]:
# View UMAP
plt.scatter(umap[:,0], umap[:,1], alpha=.1, s=1)
plt.scatter(umap[ind_outliers,0], umap[ind_outliers,1], alpha=.1, s=1)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.title(f'Particles with ||z|| > {thresh}')

In [ ]:
# Assign variables for viz/saving cells at the end of the notebook
ind_selected = ind_outliers
ind_selected_not = ind_outliers_not

# Interactive visualization

Interactive visualization of the latent encodings for the trained model. Each point represents a particle image of the dataset. The hover text includes the index of the image in the particle stack. 

In [ ]:
# Load data into a pandas dataframe on a per-particle basis
ntilts = config['dataset_args']['ntilts']
df_train = analysis.load_dataframe(z=z,
                                  pc=pc, 
                                  euler=euler[::ntilts],
                                  trans=trans[::ntilts], 
                                  kmeans_labels=kmeans_labels,
                                  gmm_labels=gmm_labels,
                                  umap=umap,
                                  df1=ctf_params[::ntilts,2],
                                  df2=ctf_params[::ntilts,3],
                                  dfang=ctf_params[::ntilts,4],
                                  phase=ctf_params[::ntilts,8],
                                  znorm=znorm)

In [ ]:
# merge training output dataframe with M s
df_train.reset_index(drop=True, inplace=True)
df_mstar.reset_index(drop=True, inplace=True)

df_merged = pd.concat( [df_train, df_mstar], axis=1) 

In [ ]:
pd.set_option('display.max_columns', None)
df_merged.head()

### Interactive selection

The next two cells contain helper code to select particles using an interactive lasso tool. 

1. In the first cell, select points with the lasso tool. The table widget is dynamically updated with the most recent selection's indices. 
2. Then once you've finalized your selection, **run the next cell** to save the particle indices for downstream analysis/viz.

(Double click to clear selection)

In [ ]:
widget, fig, ind_table = analysis.ipy_plot_interactive(df_train, opacity=0.8)
VBox((widget,fig,ind_table))

In [ ]:
ind_selected = ind_table.data[0].cells.values[0] # save table values
ind_selected = np.array(ind_selected)
ind_selected_not = invert_selection(range(len(df_train)), ind_selected)

df_merged.loc[:,'ind_selected'] = 0
df_merged.loc[ind_selected, 'ind_selected'] = 1

print('Selected indices:')
print(ind_selected)
print('Number of selected points:')
print(len(ind_selected))
print('Number of unselected points:')
print(len(ind_selected_not))

In [ ]:
# View PCA
plt.scatter(pc[:,0], pc[:,1], alpha=.1, s=1)
plt.scatter(pc[ind_selected,0], pc[ind_selected,1], alpha=.1, s=1)
plt.xlabel('PC1 ({:.2f})'.format(pca.explained_variance_ratio_[0]))
plt.ylabel('PC2 ({:.2f})'.format(pca.explained_variance_ratio_[1]))

In [ ]:
# View umap
plt.scatter(umap[:,0], umap[:,1], alpha=.1, s=1)
plt.scatter(umap[ind_selected,0], umap[ind_selected,1], alpha=.1, s=1)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

# View particles

## View images from selected particles

In [ ]:
particle_ind = ind_selected # or set to custom selection

In [ ]:
# Choose 9 particles to view at random
if len(particle_ind) > 9:
    ind_subset9 = np.random.choice(particle_ind, 9, replace=False)
else: 
    ind_subset9 = particle_ind
print(ind_subset9)

In [ ]:
p = [particles[ii].get() for ii in ind_subset9]
_ = analysis.plot_projections(p, ind_subset9)

plt.figure()
plt.scatter(umap[:,0], umap[:,1], alpha=.1, s=1)
plt.scatter(umap[ind_subset9,0], umap[ind_subset9,1], color='k')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

## View particle distributions in tomogram context

In [ ]:
tomo_list = list_paths_by_extension(
    '', # USER INPUT: INSERT ABSOLUTE PATH TO (DECONVOLVED) TOMOGRAMS
    '.mrc')
tomo_list

In [ ]:
tomo_star_mappings = {f'{i:05d}_10.00Apx.mrc' : f'{i:05d}.tomostar' for i in range(254, 319)} # USER INPUT: PROVIDE TOMOGRAM.MRC : TOMOGRAM.TOMOSTAR MAPPINGS
tomo_star_mappings

In [ ]:
analysis.interactive_tomo_ptcls(df_merged, tomo_list, tomo_star_mappings)

# Save selection

Save the particle indices for the selected (`ind_selected`) and unselected particles (`ind_selected_not`) as a .pkl file for downstream processing in cryoDRGN or with other tools.

Rename the paths as desired. Note that the indices will be automatically converted if the current cryoDRGN training run has already been filtered (`ind_orig` loaded in an earlier cell).

In [ ]:
# Set selection as either the kept or bad particles (for file naming purposes)
ind_keep = ind_selected # or ind_selected_not
ind_bad = ind_selected_not # or ind_selected

In [ ]:
# View PCA
plt.scatter(pc[:,0], pc[:,1], alpha=.1, s=1)
plt.scatter(pc[ind_keep,0], pc[ind_keep,1], alpha=.1, s=1)
plt.xlabel('PC1 ({:.2f})'.format(pca.explained_variance_ratio_[0]))
plt.ylabel('PC2 ({:.2f})'.format(pca.explained_variance_ratio_[1]))

In [ ]:
# View UMAP
plt.scatter(umap[:,0], umap[:,1], alpha=.1, s=1)
plt.scatter(umap[ind_keep,0], umap[ind_keep,1], alpha=.1, s=1)
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

In [ ]:
print('Kept particle indices:')
print(ind_keep)
print('Number of kept particles:')
print(len(ind_keep))
print('Number of bad particles:')
print(len(ind_bad))